<a href="https://colab.research.google.com/github/Durmiand/LoggiBUD-Challenge/blob/main/Aula_5_VRP_din%C3%A2mico_no_LoggiBUD_Parte_1_Solu%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código. 

Para isso, basta seguir os passos:

File -> Save a copy in Drive



---

In [ ]:
!git clone https://github.com/loggi/loggibud 
%cd /content/loggibud/

# Instale as dependências do projeto
!pip install poetry
!poetry install
# Se você estiver executando esse script localmente, não precisa dos dois comandos abaixo
!poetry export -f requirements.txt --without-hashes --output requirements.txt 
!pip install -r requirements.txt

# Verifique se tudo funcionou executando os testes
!poetry run pytest -s -v tests/

# Baixe os dados compilados
!wget -nc https://loggibud.s3.amazonaws.com/dataset.zip
!unzip -n dataset.zip

# Verifique que a pasta `data/` agora não está mais vazia
!ls data/

Cloning into 'loggibud'...
remote: Enumerating objects: 938, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 938 (delta 36), reused 45 (delta 27), pack-reused 871
Receiving objects: 100% (938/938), 6.40 MiB | 30.21 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/loggibud
     |████████████████████████████████| 174kB 6.4MB/s 
     |████████████████████████████████| 7.2MB 9.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 430kB 42.5MB/s 
     |████████████████████████████████| 348kB 36.2MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 3.2MB 33.8MB/s 
  Found existing installation: importlib-metadata 4.5.0
    Uninstalling importlib-metadata-4.5.0:
      Successfully uninstalled importlib-metadata-4.5.0
Creating virtualenv loggibud-3SbdMl6d-py3.7 in /root

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /root/.cache/pypoetry/virtualenvs/loggibud-3SbdMl6d-py3.7/bin/python
cachedir: .pytest_cache
rootdir: /content/loggibud
collected 7 items / 1 skipped / 6 selected                                     

tests/v1/test_data_conversion.py::test_can_create_proper_tsplib_from_instance PASSED
tests/v1/test_distances.py::test_great_circle_distance PASSED
tests/v1/test_distances.py::test_great_circle_route_distance PASSED
tests/v1/test_task1_baselines.py::test_ortools_solver PASSED
tests/v1/test_task1_baselines.py::test_lkh_solver PASSED
100% 158/158 [00:00<00:00, 2494.02it/s]
PASSED
100% 158/158 [00:00<00:00, 64894.25it/s]
PASSED

========================= 7 passed, 1 skipped in 8.46s =========================
--2021-06-22 01:18:47--  https://loggibud.s3.amazonaws.com/dataset.zip
Resolving loggibud.s3.amazonaws.com (loggibud.s3.amazonaws.com

In [ ]:
# Arquivo de configuração para acessar o OSRM
from loggibud.v1.distances import OSRMConfig


osrm_config = OSRMConfig(host="http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com")
osrm_config

OSRMConfig(host='http://ec2-34-222-175-250.us-west-2.compute.amazonaws.com', timeout_s=600)

# Exercício 1

Comecemos importando a instância desejada:

In [ ]:
from loggibud.v1.types import CVRPInstance

file_path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
problem = CVRPInstance.from_file(file_path)
problem

CVRPInstance(name='cvrp-0-df-0', region='df-0', origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), vehicle_capacity=180, deliveries=[Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10), Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10), Delivery(id='54e598b20ffa2707d01b1ac00942793f', point=Point(lng=-47.755018806591394, lat=-15.651207309372888), size=7), Delivery(id='64812681ba8c280649f764b9fb40936a', point=Point(lng=-47.758785390289965, lat=-15.648706444367969), size=3), Delivery(id='c17ea5bb3bdb1346ea358516cba81e5c', point=Point(lng=-47.75284167302011, lat=-15.66047286919706), size=10), Delivery(id='fb1b2bc2ac3783706de766e715cb9d01', point=Point(lng=-47.74788198106806, lat=-15.663388754816273), size=3), Delivery(id='c667f779151bd4e23e91d3b4659ffd94', point=Point(lng=-47.75836162961292, lat=-15.648355505737312), size=5), Delivery(id='7b315f

In [ ]:
# Veja estas entregas no mapa
from loggibud.v1.plotting.plot_instance import plot_cvrp_instance


plot_cvrp_instance(problem)

Aqui está o código completo do nosso algoritmo trivial desenvolvido na aula:

In [ ]:
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle


def solve_vrp_naive_algorithm(problem):
    vehicles = []  # lista com todos os veículos
    vehicle_deliveries = []  # inicializa o primeiro veículo vazio
    for delivery in problem.deliveries:
        if (
            _compute_vehicle_volume(vehicle_deliveries) + delivery.size
            <= problem.vehicle_capacity
        ):
            vehicle_deliveries.append(delivery)
        else:
            vehicle_solution = _construct_vehicle(problem, vehicle_deliveries)
            vehicles.append(vehicle_solution)
            # Inicializa novo veículo com o pacote atual
            vehicle_deliveries = [delivery]

    # Complete a rota do veículo final que sobrou
    if vehicle_deliveries:
        vehicle_solution = _construct_vehicle(problem, vehicle_deliveries)
        vehicles.append(vehicle_solution)

    # Ao final, retorne uma variável do tipo `CVRPSolution`
    return CVRPSolution(name=problem.name, vehicles=vehicles)


def _compute_vehicle_volume(deliveries):
    volume = 0
    for delivery in deliveries:
        volume += delivery.size
    return volume


def _construct_vehicle(problem, vehicle_deliveries):
    return CVRPSolutionVehicle(
        origin=problem.origin, deliveries=vehicle_deliveries
    )


In [ ]:
# Resolvendo para a instância pedida
solution = solve_vrp_naive_algorithm(problem)
solution

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10), Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10), Delivery(id='54e598b20ffa2707d01b1ac00942793f', point=Point(lng=-47.755018806591394, lat=-15.651207309372888), size=7), Delivery(id='64812681ba8c280649f764b9fb40936a', point=Point(lng=-47.758785390289965, lat=-15.648706444367969), size=3), Delivery(id='c17ea5bb3bdb1346ea358516cba81e5c', point=Point(lng=-47.75284167302011, lat=-15.66047286919706), size=10), Delivery(id='fb1b2bc2ac3783706de766e715cb9d01', point=Point(lng=-47.74788198106806, lat=-15.663388754816273), size=3), Delivery(id='c667f779151bd4e23e91d3b4659ffd94', point=Point(lng=-47.75836162961292, lat=-15.648355505737312), size=5), Delivery(id='7b315f77e3dfd

In [ ]:
print(f"A solução contém {len(solution.vehicles)} veículos")

A solução contém 33 veículos


In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution


total_distance = evaluate_solution(problem, solution, config=osrm_config)
print(f"O percurso completo leva {total_distance} km")

O percurso completo leva 4210.638 km


Vamos visualizar estas rotas

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)

Claramente temos aqui uma bagunça. Portanto, vamos focar nossa atenção nas três primeiras rotas apenas:

In [ ]:
plot_cvrp_solution(solution, route_indices_to_plot=[0, 1, 2])

Observe a quantidade de zigue-zagues obtidos em cada rota. Apesar de nossa solução ser simples e rápida, ela claramente seria rejeitada por qualquer empresa séria por causar este tipo de ineficiência.

Experimente depois ver como ficam as rotas em rua.

# Exercício 2

Aqui está o código completo do solver trivial com o TSP

In [ ]:
import numpy as np
from ortools.constraint_solver import pywrapcp

from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle


def solve_vrp_naive_algorithm_with_tsp(problem):
    vehicles = []  # lista com todos os veículos
    vehicle_deliveries = []  # inicializa o primeiro veículo vazio
    for delivery in problem.deliveries:
        if (
            _compute_vehicle_volume(vehicle_deliveries) + delivery.size
            <= problem.vehicle_capacity
        ):
            vehicle_deliveries.append(delivery)
        else:
            vehicle_solution = _construct_vehicle(problem, vehicle_deliveries)
            vehicles.append(vehicle_solution)
            # Inicializa novo veículo com o pacote atual
            vehicle_deliveries = [delivery]

    # Complete a rota do veículo final que sobrou
    if vehicle_deliveries:
        vehicle_solution = _construct_vehicle(problem, vehicle_deliveries)
        vehicles.append(vehicle_solution)

    # Ao final, retorne uma variável do tipo `CVRPSolution`
    return CVRPSolution(name=problem.name, vehicles=vehicles)


def _compute_vehicle_volume(deliveries):
    volume = 0
    for delivery in deliveries:
        volume += delivery.size
    return volume


def _construct_vehicle(problem, vehicle_deliveries):
    distance_matrix = _compute_distance_matrix(problem, vehicle_deliveries)
    ordered_indices, _ = solve_tsp_ortools(distance_matrix)

    # `ordered_indices` tem o formato `[0, 4, 3, 1, ..., 0]`. Precisamos
    # remover a origem do início e do final e ordenar a variável
    # `vehicle_deliveries` usando estes índices
    ordered_vehicle_deliveries = []
    for ordered_index in ordered_indices[1:-1]:
        ordered_vehicle_deliveries.append(
            vehicle_deliveries[ordered_index - 1]
        )

    return CVRPSolutionVehicle(
        origin=problem.origin, deliveries=ordered_vehicle_deliveries
    )


def _compute_distance_matrix(problem, vehicle_deliveries):
    # Os pontos do problema consistem na origem mais as entregas em
    # `vehicle_deliveries`
    points = [problem.origin]
    for delivery in vehicle_deliveries:
        points.append(delivery.point)

    # Configuração com o servidor para os alunos
    return calculate_distance_matrix_m(points, config=osrm_config)


def solve_tsp_ortools(distance_matrix):
    n = distance_matrix.shape[0]  # número de nós do problema
    num_vehicles = 1  # número de veículos (no nosso caso, apenas um)
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro 
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constroi a rota final
    route = []
    index = routing.Start(0)
    node = manager.IndexToNode(index)
    route.append(node)

    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)
        route.append(node)
    
    return route, solution.ObjectiveValue()

Resolvendo o mesmo problema de antes, temos (observe que, graças ao TSP, a solução pode levar um pouco mais de tempo):

In [ ]:
from loggibud.v1.types import CVRPInstance

file_path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
problem = CVRPInstance.from_file(file_path)
solution = solve_vrp_naive_algorithm_with_tsp(problem)
solution

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='64812681ba8c280649f764b9fb40936a', point=Point(lng=-47.758785390289965, lat=-15.648706444367969), size=3), Delivery(id='c667f779151bd4e23e91d3b4659ffd94', point=Point(lng=-47.75836162961292, lat=-15.648355505737312), size=5), Delivery(id='54e598b20ffa2707d01b1ac00942793f', point=Point(lng=-47.755018806591394, lat=-15.651207309372888), size=7), Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10), Delivery(id='eebf0d97e6829796d7c012d3d0765647', point=Point(lng=-47.75482460546861, lat=-15.65435446002705), size=1), Delivery(id='5a1d00fc8dc1ee2a901740b55cdd36aa', point=Point(lng=-47.76340110906013, lat=-15.668974870973102), size=1), Delivery(id='ed0993f8cc70d998342f38ee827176dc', point=Point(lng=-47.7496622016347, lat=-15.65879313293694), size=10), Delivery(id='c17ea5bb3bdb13

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution


total_distance = evaluate_solution(problem, solution, config=osrm_config)
print(f"O percurso completo leva {total_distance} km")

O percurso completo leva 2292.8135 km


Observe como a distância total é reduzida para quase a metade ao utilizar o TSP.

In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)

Observe o poder do TSP ao converter as rotas em algo muito menos bagunçado comparado com antes.

Experimente comparar os dois solvers em outras instâncias para ver as melhoras obtidas.

# Exercício 3

In [ ]:
import numpy as np

from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle


def solve_vrp_constructive_algorithm_with_tsp(problem):
    vehicles = []  # lista com todos os veículos
    for delivery in problem.deliveries:
        available_vehicles = _filter_available_vehicles(
            problem, vehicles, delivery
        )
        vehicle_number = _compute_delivery_distance_from_vehicles(
            problem, available_vehicles, delivery
        )

        # Se `vehicle_number` == -1, criamos um novo veículo com esta entrega
        if vehicle_number < 0:
            vehicles.append([delivery])
        # Senão, adicione a entrega no `vehicle_number`
        else:
            available_vehicles[vehicle_number].append(delivery)

    # Ao final construa rotas com cada veículo
    vehicle_solutions = []
    for vehicle_deliveries in vehicles:
        vehicle_solution = _construct_vehicle(problem, vehicle_deliveries)
        vehicle_solutions.append(vehicle_solution)

    return CVRPSolution(name=problem.name, vehicles=vehicle_solutions)


def _filter_available_vehicles(problem, vehicles, delivery):
    """
    Filtra os veículos com volume restante suficiente para caber um novo pacote
    """
    available_vehicles = []
    for vehicle_deliveries in vehicles:
        if (
            _compute_vehicle_volume(vehicle_deliveries) + delivery.size
            <= problem.vehicle_capacity
        ):
            available_vehicles.append(vehicle_deliveries)

    return available_vehicles


def _compute_vehicle_volume(deliveries):
    volume = 0
    for delivery in deliveries:
        volume += delivery.size
    return volume


def _compute_delivery_distance_from_vehicles(
    problem, available_vehicles, delivery
):
    """
    Calcula a distância de `delivery` à origem e a cada pacote dos veículos
    disponíveis e retorna o veículo mais próximo.
    """

    # Inicializa `points` com a entrega atual e a origem
    # Inicializa também `vehicle_numbers` com -1 já que estes pontos não
    # percentem a nenhum veículos
    points = [delivery.point, problem.origin]
    vehicle_numbers = [-1, -1]

    # Extrai os pontos de cada veículos
    for i, vehicle_deliveries in enumerate(available_vehicles):
        for delivery in vehicle_deliveries:
            points.append(delivery.point)
            vehicle_numbers.append(i)

    # Calcula a matriz de distâncias
    distance_matrix = calculate_distance_matrix_m(points, config=osrm_config)

    # Determina o ponto mais próximo da entrega atual
    distance_matrix[0, 0] = np.inf
    imin = distance_matrix[0].argmin()
    return vehicle_numbers[imin]


def _construct_vehicle(problem, vehicle_deliveries):
    distance_matrix = _compute_distance_matrix(problem, vehicle_deliveries)
    ordered_indices, _ = solve_tsp_ortools(distance_matrix)

    # `ordered_indices` tem o formato `[0, 4, 3, 1, ..., 0]`. Precisamos
    # remover a origem do início e do final e ordenar a variável
    # `vehicle_deliveries` usando estes índices
    ordered_vehicle = []
    for ordered_index in ordered_indices[1:-1]:
        ordered_vehicle.append(vehicle_deliveries[ordered_index - 1])

    return CVRPSolutionVehicle(
        origin=problem.origin, deliveries=ordered_vehicle
    )


def _compute_distance_matrix(problem, vehicle_deliveries):
    # Os pontos do problema consistem na origem mais as entregas em
    # `vehicle_deliveries`
    points = [problem.origin]
    for delivery in vehicle_deliveries:
        points.append(delivery.point)

    # Configuração com o servidor para os alunos
    return calculate_distance_matrix_m(points, config=osrm_config)

In [ ]:
from loggibud.v1.types import CVRPInstance

file_path = "./data/cvrp-instances-1.0/train/df-0/cvrp-0-df-0.json"
problem = CVRPInstance.from_file(file_path)
solution = solve_vrp_constructive_algorithm_with_tsp(problem)
solution

CVRPSolution(name='cvrp-0-df-0', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='64812681ba8c280649f764b9fb40936a', point=Point(lng=-47.758785390289965, lat=-15.648706444367969), size=3), Delivery(id='c667f779151bd4e23e91d3b4659ffd94', point=Point(lng=-47.75836162961292, lat=-15.648355505737312), size=5), Delivery(id='54e598b20ffa2707d01b1ac00942793f', point=Point(lng=-47.755018806591394, lat=-15.651207309372888), size=7), Delivery(id='c7220154adc7a3def8f0b2b8a42677a9', point=Point(lng=-47.75887552060412, lat=-15.651440380492554), size=10), Delivery(id='eebf0d97e6829796d7c012d3d0765647', point=Point(lng=-47.75482460546861, lat=-15.65435446002705), size=1), Delivery(id='5a1d00fc8dc1ee2a901740b55cdd36aa', point=Point(lng=-47.76340110906013, lat=-15.668974870973102), size=1), Delivery(id='c17ea5bb3bdb1346ea358516cba81e5c', point=Point(lng=-47.75284167302011, lat=-15.66047286919706), size=10), Delivery(id='ed0993f8cc70d

In [ ]:
from loggibud.v1.eval.task1 import evaluate_solution


total_distance = evaluate_solution(problem, solution, config=osrm_config)
print(f"O percurso completo leva {total_distance} km")

O percurso completo leva 2222.0534 km


In [ ]:
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution


plot_cvrp_solution(solution)

Este solver é claramente mais demorado, o que é esperado dada a quantidade de operações feitas para cada pacote. A melhoria na solução parece também não compensar este aumento no processamento.

Será que isto é suficiente para dizer que ele não vale a pena?

Resolva os exercícios suplementares e experimente ver o quão robusto ele é quando a ordem dos pacotes é alterada.